In [2]:
import os
import sys
import numpy as np
import random

from pommerman.agents import SimpleAgent, RandomAgent, PlayerAgent, BaseAgent
from pommerman.configs import ffa_v0_fast_env,  one_vs_one_env
from pommerman.envs.v0 import Pomme
from pommerman.characters import Bomber
from pommerman import utility

from pommerman.constants import Action

In [48]:
def rotate_obs(obs):
    """
    Rotate an agents observation by 90 degrees counter-clockwise
    """
    rotated_obs = obs.copy()
    rotated_obs['board'] = np.rot90(obs['board'])
    rotated_obs['bomb_blast_strength'] = np.rot90(obs['bomb_blast_strength']) 
    rotated_obs['bomb_life'] = np.rot90(obs['bomb_life']) 
    rotated_obs['bomb_moving_direction'] = np.rot90(obs['bomb_moving_direction']) 
    rotated_obs['flame_life'] = np.rot90(obs['flame_life'])

    # rotate position of agent
    rotated_obs['position'] = (-obs['position'][1]+10, obs['position'][0]) 
    return rotated_obs

def rotate_action(action):
    """
    Rotate an agents actions by 90 degrees counter-clockwise
    """
    if action == Action.Stop.value or Action.Bomb.value:
        action_rotated = action
    elif action == Action.Up.value:
        action_rotated = Action.Left.value
    elif action == Action.Down.value:
        action_rotated = Action.Right.value
    elif action == Action.Left.value:
        action_rotated = Action.Down.value
    elif action == Action.Right.value:
        action_rotated = Action.Up.value
    return action_rotated

def mirror_horizontal_obs(obs):
    """
    Mirror an agents observation horizontally
    """
    mirrored_obs = obs.copy()
    mirrored_obs['board'] = np.flip(obs['board'], 1)
    mirrored_obs['bomb_blast_strength'] = np.flip(obs['bomb_blast_strength'], 1) 
    mirrored_obs['bomb_life'] = np.flip(obs['bomb_life'], 1) 
    mirrored_obs['bomb_moving_direction'] = np.flip(obs['bomb_moving_direction'], 1) 
    mirrored_obs['flame_life'] = np.flip(obs['flame_life'], 1)

    # mirror position of agent
    mirrored_obs['position'] = (10-obs['position'][0], obs['position'][1]) 
    return mirrored_obs

def mirror_horizontal_action(action):
    '''
    Mirror an agents observation horizontally
    '''
    if action == Action.Stop.value or Action.Bomb.value or action == Action.Up.value or action == Action.Down.value:
        action_mirrored = action
    elif action == Action.Left.value:
        action_mirrored = Action.Right.value
    elif action == Action.Right.value:
        action_mirrored = Action.Left.value
    return action_mirrored

def mirror_vertical_obs(obs):
    """
    Mirror an agents transition vertically
    """
    mirrored_obs = obs.copy()
    mirrored_obs['board'] = np.flip(obs['board'], 0)
    mirrored_obs['bomb_blast_strength'] = np.flip(obs['bomb_blast_strength'], 0) 
    mirrored_obs['bomb_life'] = np.flip(obs['bomb_life'], 0) 
    mirrored_obs['bomb_moving_direction'] = np.flip(obs['bomb_moving_direction'], 0) 
    mirrored_obs['flame_life'] = np.flip(obs['flame_life'], 0)

    # mirror position of agent
    mirrored_obs['position'] = (obs['position'][0], 10-obs['position'][1]) 
    return mirrored_obs

def mirror_vertical_action(action):
    """
    Mirror an agents actions vertically
    """
    if action == Action.Stop.value or Action.Bomb.value or action == Action.Left.value or action == Action.Right.value:
        action_mirrored = action
    elif action == Action.Up.value:
        action_mirrored = Action.Down.value
    elif action == Action.Down.value:
        action_mirrored = Action.Up.value
    return action_mirrored

In [49]:
rotate_obs(obs[0])

{'alive': [10, 11],
 'board': array([[ 0,  2,  2,  2,  0,  2,  2,  2,  2,  0,  0],
        [ 0,  2,  1,  0,  1,  1,  2,  0,  0, 11,  0],
        [ 2,  0,  0,  1,  1,  1,  1,  0,  0,  0,  2],
        [ 0,  1,  0,  1,  2,  1,  1,  0,  0,  0,  2],
        [ 0,  2,  2,  0,  2,  2,  0,  1,  1,  2,  2],
        [ 0,  2,  1,  1,  1,  0,  2,  1,  1,  1,  2],
        [ 2,  1,  2,  1,  0,  1,  2,  2,  1,  1,  0],
        [ 0,  0,  1,  0,  1,  1,  0,  1,  1,  0,  2],
        [ 1,  0,  0,  1,  2,  1,  2,  0,  0,  1,  2],
        [ 0, 10,  0,  0,  1,  2,  2,  1,  0,  2,  2],
        [ 0,  0,  1,  0,  2,  0,  0,  0,  2,  0,  0]], dtype=uint8),
 'bomb_blast_strength': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.

In [50]:
from data_augmentation import DataAugmentor_v1

aug = DataAugmentor_v1()
aug.augment(obs[0], 4, 1, obs[0], False)
# aug.rotate_obs(obs[0])

TypeError: rotate_obs() takes 1 positional argument but 2 were given

In [34]:
nobs = obs[0]

mirrored_nobs = mirror_vertical_obs(nobs)
# np.flip(obs[0]['board'], 0)

print(mirrored_nobs['board'])
print(mirrored_nobs['position'])



[[ 0  2  2  2  0  2  2  2  2  0  0]
 [ 0  2  1  0  1  1  2  0  0 11  0]
 [ 2  0  0  1  1  1  1  0  0  0  2]
 [ 0  1  0  1  2  1  1  0  0  0  2]
 [ 0  2  2  0  2  2  0  1  1  2  2]
 [ 0  2  1  1  1  0  2  1  1  1  2]
 [ 2  1  2  1  0  1  2  2  1  1  0]
 [ 0  0  1  0  1  1  0  1  1  0  2]
 [ 1  0  0  1  2  1  2  0  0  1  2]
 [ 0 10  0  0  1  2  2  1  0  2  2]
 [ 0  0  1  0  2  0  0  0  2  0  0]]
(1, 9)


In [39]:
array = np.array([[1,2,3],
                [4,5,6],
                [7,8,9]])

a = np.flip(array, 0)
np.rot90(a)

array([[9, 6, 3],
       [8, 5, 2],
       [7, 4, 1]])

In [4]:
# Instantiate the environment
config = ffa_v0_fast_env()
env = Pomme(**config["env_kwargs"])

# Add four random agents
agents = {}
for agent_id in range(2):
    agents[agent_id] = RandomAgent(config["agent"](agent_id, config["game_type"]))
env.set_agents(list(agents.values()))
env.set_init_game_state(None)

In [5]:
# Seed and reset the environment
env.seed(0)
obs = env.reset()

In [4]:
a = (1,2,3)
a[0]

1